Read <http://docs.continuum.io/numbapro/quickstart.html>

In [5]:
import mkl
from numbapro import int32, float32

In [7]:
print(int32[:])           # 1D int32 array
print(float32[:,:])       # 2D float32 array
print(int32[:,:,:,:])     # 4D int32 array

array(int32, 1d, A, nonconst)
array(float32, 2d, A, nonconst)
array(int32, 4d, A, nonconst)


In [8]:
from numbapro import void, int32, float32, complex64
print(complex64(int32, float32, complex64))
print(float32())                             # no arguments
print(void(float32))                         # return nothing
print(void(float32[:], int32[:]))

complex64(int32, float32, complex64)
float32()
none(float32)
none(array(float32, 1d, A, nonconst), array(int32, 1d, A, nonconst))


In [9]:
from numbapro import jit, int32, float32, complex64

@jit(complex64(int32, float32, complex64), target="cpu")
def bar(a, b, c):
   return a + b  * c

@jit(complex64(int32, float32, complex64)) # target kwarg defaults to "cpu"
def foo(a, b, c):
   return a + b  * c


print(foo)
print(foo(1, 2.0, 3.0j))

CPUOverloaded(<function foo at 0x10f2aeea0>)
(1+6j)


In [10]:
from numbapro import autojit

@autojit(target="cpu")
def bar(a, b, c):
    return a + b * c

@autojit                    # target kwarg defaults to "cpu"
def foo(a, b, c):
    return a + b * c

print(foo)
print(foo(1, 2.0, 3j))

CPUOverloaded(<function foo at 0x1007dc048>)
(1+6j)


In [12]:
from numbapro import vectorize
from numpy import arange

@vectorize(['float32(float32, float32)'], target='cpu') # default to 'cpu'
def add2(a, b):
    return a + b

X = arange(10, dtype='float32')
Y = X * 2
print(add2(X, Y))
print(add2.reduce(X))

[  0.   3.   6.   9.  12.  15.  18.  21.  24.  27.]
45.0


In [14]:
from numbapro import guvectorize
from numpy import arange

@guvectorize(['void(float64[:,:], float64[:,:], float64[:,:])'],
             '(m,n),(n,p)->(m,p)')
def matmul(A, B, C):
    m, n = A.shape
    n, p = B.shape
    for i in range(m):
        for j in range(p):
            C[i, j] = 0
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

w = 2
A = arange(w**2).reshape(w, w)
B = arange(w**2).reshape(w, w)
C = matmul(A, B)
print("A:\n%s" % A)
print("B:\n%s" % B)
print("C:\n%s" % C)

A:
[[0 1]
 [2 3]]
B:
[[0 1]
 [2 3]]
C:
[[  2.   3.]
 [  6.  11.]]


In [15]:
from numbapro import guvectorize
from numpy import zeros, arange

@guvectorize(['void(int32[:], int32[:])'], '(n)->()')
def sum_row(inp, out):
    """
    Sum every row

    function type: two arrays
                   (note: scalar is represented as an array of length 1)
    signature: n elements to scalar
    """
    tmp = 0.
    for i in range(inp.shape[0]):
        tmp += inp[i]
    out[0] = tmp

inp = arange(15, dtype='int32').reshape(5, 3)
print(inp)

# implicit output array
out = sum_row(inp)
print('imp: %s' % out)

# explicit output array
explicit_out = zeros(5, dtype='int32')
sum_row(inp, out=explicit_out)
print('exp: %s' % explicit_out)

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]
imp: [ 3 12 21 30 39]
exp: [ 3 12 21 30 39]
